# Quantum Circuits
Quantum computers can only use a specific set of gates (universal gate set). Given the entanglers and their amplitudes found in Step 3, one can find corresponding representation of these operators in terms of elementary gates using the following procedure.

In [14]:
import tequila as tq
from utility import *
import time

In [15]:
#Focus on one molecule
molecules=['h2']
methods=['cisd']
basies=['sto-3g']
qubit_transfms=['bk'] 

In [16]:
R=0.65 #bond_length

for mol in molecules:

    for basis in basies:

        for qubit_transf in qubit_transfms:
                    
            start=time.time()
            
            H = get_qubit_hamiltonian(mol, geometry=R, basis=basis, multiplicity=3, qubit_transf=qubit_transf)
            print("Take a look at H!")
            
            xyz_data = get_molecular_data(mol, geometry=R, xyz_format=True)
            Htq = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

            E_cisd=obtain_PES(mol, [R], basis, 'cisd')
            print('cisd'.upper()," ground state energy is:",E_cisd)
            
            n_spin_orbitals=2*Htq.n_orbitals
            n_electrons=Htq.n_electrons
            print("Numbers of electrons:",n_electrons,
                  "\nUsing ",qubit_transf.upper()," in ",basis.upper(),"basis.",
                  "\nNumber of spin-orbitals (qubits):",n_spin_orbitals,
                  "\nNumber of Hamiltonian terms:",len(H.terms)) 

            H_eff=taper_hamiltonian(H, n_spin_orbitals, n_electrons, qubit_transf)
            print("Number of terms in Taper effective Hamiltonian (H_eff):",len(H_eff.terms))

            
            if mol=='h2':
                # Building the matrix representation of the effective Hamiltonian
                I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])
                H_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

                # Obtain the eigenvalues
                eigvals, _ = np.linalg.eigh(H_matrix)
                print("The eigenvalues of the effective Hamiltonian:{}".format(eigvals))
            else:
                print("H_matrix for qubit-tapering technique not yet implemeneted for ",mol.upper())

            print(mol.upper(),"Hamiltonian using ",qubit_transf.upper()," in basis:",
                  basis.upper(),"generated for:",time.time()-start," sec\n")
            
            

Take a look at H!
E = -1.129904784381681 Eh
CISD  ground state energy is: [-1.12990478]
Numbers of electrons: 2 
Using  BK  in  STO-3G basis. 
Number of spin-orbitals (qubits): 4 
Number of Hamiltonian terms: 15
Number of terms in Taper effective Hamiltonian (H_eff): 3
The eigenvalues of the effective Hamiltonian:[-1.10115031  0.03904763]
H2 Hamiltonian using  BK  in basis: STO-3G generated for: 1.5354807376861572  sec



In [17]:
qwc_list = get_qwc_group(H)
print('Fragments 1: \n{}\n'.format(qwc_list[0]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
0.03775110394645728 [] +
0.04407961290255181 [Y0 Z1 Y2] +
0.04407961290255181 [Y0 Z1 Y2 Z3] +
0.17297610130745106 [Z1] +
0.17866777775953419 [Z1 Z3]

Fragments 2:
0.04407961290255181 [X0 Z1 X2] +
0.04407961290255181 [X0 Z1 X2 Z3]

Number of fragments: 3


In [18]:
comm_groups = get_commuting_group(H)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])
print(comm_groups[2])

Number of mutually commuting fragments: 2
The first commuting group
0.03775110394645728 [] +
0.04407961290255181 [X0 Z1 X2] +
0.04407961290255181 [X0 Z1 X2 Z3] +
0.04407961290255181 [Y0 Z1 Y2] +
0.04407961290255181 [Y0 Z1 Y2 Z3] +
0.16992097848261525 [Z0 Z1 Z2] +
0.16992097848261525 [Z0 Z1 Z2 Z3] +
0.12584136558006345 [Z0 Z2] +
0.12584136558006345 [Z0 Z2 Z3] +
0.17297610130745106 [Z1] +
0.17866777775953419 [Z1 Z3]
0.1860164888623057 [Z0] +
0.1860164888623056 [Z0 Z1] +
-0.26941693141632106 [Z1 Z2 Z3] +
-0.26941693141632117 [Z2]


In [22]:
%%time
#One has to loop over all groups

allz_list = []  # store the allz from all iterations
for i in range(1,len(comm_groups)+1):
    uqwc = get_qwc_unitary(comm_groups[i])
    qwc = remove_complex(uqwc * comm_groups[i] * uqwc)

    uz = get_zform_unitary(qwc)
    print("Checking whether U * U^+ is identity: {}".format(uz * uz))

    allz = remove_complex(uz * qwc * uz)
    allz_list.append(allz)
    print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
0.03775110394645723 [] +
0.1258413655800633 [Z0] +
0.16992097848261511 [Z0 Z1] +
-0.044079612902551774 [Z0 Z1 Z2] +
-0.044079612902551774 [Z0 Z1 Z2 Z3] +
0.16992097848261511 [Z0 Z1 Z3] +
0.1258413655800633 [Z0 Z3] +
0.1729761013074509 [Z1] +
0.044079612902551774 [Z1 Z2] +
0.044079612902551774 [Z1 Z2 Z3] +
0.17866777775953396 [Z1 Z3]
Checking whether U * U^+ is identity: 0.9999999999999996 []

The all-z form of qwc fragment:
0.18601648886230557 [Z0] +
0.1860164888623054 [Z0 Z1] +
-0.26941693141632056 [Z1 Z2 Z3] +
-0.2694169314163207 [Z2]
CPU times: user 118 ms, sys: 0 ns, total: 118 ms
Wall time: 116 ms


Save the info in allz_list into the file `./hamiltonians/test.inp`, which has the same format as all the other files in that directoy.

In [68]:
n = allz_list[0].many_body_order() # 4

output = {}
for allz in allz_list:
    for k,v in allz.terms.items():
        key = ['e'] * n
        for spin in k:
            key[n-1-spin[0]] = 'z'
        key_str = ''.join(key)
        output[key_str] = output.get(key_str, 0) + v
        
with open('./hamiltonians/test.inp', 'w') as fh:
    fh.write("4 13 real\n")
    for k,v in output.items():
        fh.write(k + ' ' + str(v) + '\n')

Temporarily copy here some definitions from task 3 for quick testing

In [74]:
import numpy as np
import matplotlib.pyplot as plt
from common.abstract_ising import AbstractIsing
from common.ising_animator import IsingAnimator
from common.utils import *

def read_generalized_ising_hamiltonian(path):
    with open(path, "r") as f:
        f.readline()  # discard first line
        compressed_hamiltonian = [
            tuple(line.strip().split())
            for line in f.readlines()
        ]
    
    num_sites = len(compressed_hamiltonian[0][0])
    hamiltonian_terms = [np.zeros((num_sites,)*i) for i in range(num_sites+1)]

    for sites, val in compressed_hamiltonian:
        num_zs = 0
        site_nums = []
        for i, x in enumerate(sites):
            if x == 'z':
                site_nums.append(i)
                num_zs += 1

        hamiltonian_terms[num_zs][tuple(site_nums)] = float(val)

    return hamiltonian_terms


class GeneralizedIsingModel(AbstractIsing):
    def __init__(self, E0, h, J, K, L):
        self.E0 = E0[()]
        self.h = h
        self.J = J
        self.K = K
        self.L = L
        
        self.num_spins = h.shape[0]
        
        # initialize the spins randomly
        self.spins = 2 * (np.random.rand(self.num_spins) < 0.5) - 1
    
    def energy(self, spins=None):
        """
        Args:
            spins: a single spin configuration, or a list of spin configurations
            
        Returns:
            Energies corresponding to the given spin configurations
        """
        if spins is None:
            spins = self.spins
            
        if len(spins.shape) == 1:
            spins = spins[None, :]
        
        energy = self.E0 * np.ones(spins.shape[0])
        energy += np.einsum('i,bi->b', self.h, spins)
        energy += np.einsum('ij,bi,bj->b', self.J, spins, spins)
        energy += np.einsum('ijk,bi,bj,bk->b', self.K, spins, spins, spins)
        energy += np.einsum('ijkl,bi,bj,bk,bl->b', self.L, spins, spins, spins, spins)
        
        return energy
    
    def energy_diff(self, i):
        dE = 0
        for coef in [self.h, self.J, self.K, self.L]:
            tmp = 0
            for j in range(len(coef.shape)):
                tmp += np.moveaxis(coef, j, 0)[i]
            for _ in range(len(coef.shape) - 1):
                tmp = tmp.dot(self.spins)
            dE += tmp     
        dE *= -2 * self.spins[i] 
        
        return dE
        
    
    def rand_site(self):
        return (np.random.randint(self.num_spins),)
    
    def all_configurations(self):
        """Returns all possible spin configurations"""
        dim = np.arange(2 ** ising.num_spins)
        space = ((dim[:, None] & (1 << np.arange(ising.num_spins))) > 0)
        space = 2*space.astype(int) - 1
        return space

Determine the minimum energy for the hamiltonian based on the file we have just created.

In [75]:
E0, h, J, K, L = read_generalized_ising_hamiltonian("./hamiltonians/test.inp")
ising = GeneralizedIsingModel(E0, h, J, K, L)
ising.energy(ising.all_configurations()).min()

-0.93667809405896

The correct result for R=0.65 should be -1.1299047752322906

In [ ]:
# R(A)         E(Ha)
0.65      -1.1299047752322906
0.8      -1.1341476722233872
0.95      -1.1113394317141152
1.1      -1.0791929635915072
1.25      -1.0457831649744007
1.4      -1.0154682691531118
1.55      -0.9904763585526108
1.7      -0.9714267029717627
1.85      -0.9578329791835556
2.0      -0.9486411206967024
2.15      -0.9426777920320297
2.3      -0.9389223907402545
2.45      -0.9366052600789352
2.6      -0.9351960337828166
2.75      -0.9343489902069892
2.9      -0.9338457529323188
3.05      -0.9335506069059256
3.2      -0.933379979185252
3.35      -0.9332828445659485
3.5      -0.9332284072659003